# Clustering Crypto

In [56]:
# Initial imports
import pandas as pd
import hvplot.pandas
#from path import Path
from os import path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [76]:
# Load the crypto_data.csv dataset.
# Load data
re_df = pd.read_csv(r'C:\Users\earth\Desktop\class_folder\FinalProject\Resources\final_table_with_conditional.csv')
re_df.head(10)

,Date,Year,County,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,average_listing_price,total_listing_count,violent_crime_rate,30_Year_Fixed_Rate,Number_of_Schools,median_listing_price_mm,Number_of_Parks,Price_Status
0,2022-06,2022,Cherokee,362500.0,271.0,38.0,148.0,8.0,76.0,1904.0,407510.0,467.0,NaN,5.52,NaN,-0.052164,2.0,down
1,2022-06,2022,Craven,313500.0,154.0,36.0,196.0,12.0,48.0,1927.0,396716.0,581.0,NaN,5.52,NaN,0.039801,63.0,up
2,2022-06,2022,Catawba,342450.0,207.0,34.0,224.0,0.0,96.0,1923.0,521108.0,508.0,NaN,5.52,NaN,-0.042098,65.0,down
3,2022-06,2022,Gaston,330000.0,283.0,30.0,412.0,12.0,200.0,1668.0,377406.0,828.0,NaN,5.52,NaN,0.039370,99.0,up
4,2022-06,2022,Warren,275000.0,19.0,68.0,12.0,0.0,4.0,1824.0,419452.0,29.0,NaN,5.52,NaN,NaN,1.0,down
5,2022-06,2022,Lenoir,200000.0,75.0,38.0,68.0,8.0,20.0,1810.0,211563.0,143.0,NaN,5.52,NaN,0.093494,13.0,up
6,2022-06,2022,Lee,341450.0,93.0,19.0,128.0,0.0,24.0,2325.0,411072.0,242.0,NaN,5.52,NaN,0.054021,22.0,up
7,2022-06,2022,Henderson,552500.0,235.0,39.0,212.0,4.0,88.0,2167.0,862483.0,556.0,NaN,5.52,NaN,-0.022124,104.0,down
8,2022-06,2022,Beaufort,324450.0,77.0,39.0,60.0,0.0,24.0,1900.0,383134.0,152.0,NaN,5.52,NaN,0.081504,18.0,up
9,2022-06,2022,Hertford,192000.0,23.0,57.0,16.0,0.0,0.0,2052.0,245064.0,41.0,NaN,5.52,NaN,NaN,3.0,down


In [77]:
# Load the data
columns = [
    "Date", "Year", "County", "median_listing_price",
    "active_listing_count", "median_days_on_market", "new_listing_count",
    "price_increased_count", "price_reduced_count", "median_square_feet",
    "average_listing_price", "total_listing_count", "violent_crime_rate",
    "30_Year_Fixed_Rate", "Number_of_Schools", "median_listing_price_mm",
    "Number_of_Parks", "Price_Status"]

df = re_df.loc[:, columns].copy()


# Drop the null rows
df = re_df.dropna()




df.reset_index(inplace=True, drop=True)

df.head()

,Date,Year,County,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,average_listing_price,total_listing_count,violent_crime_rate,30_Year_Fixed_Rate,Number_of_Schools,median_listing_price_mm,Number_of_Parks,Price_Status
0,2018-12,2018,Wilkes,224000.0,306.0,126.0,52.0,0.0,28.0,2174.0,352753.0,340.0,243.6,4.64,53.0,-0.025663,34.0,down
1,2018-12,2018,Forsyth,245000.0,1197.0,85.0,296.0,12.0,204.0,2234.0,325987.0,1445.0,660.2,4.64,354.0,-0.019608,479.0,down
2,2018-12,2018,Surry,179900.0,145.0,114.0,28.0,0.0,24.0,1987.0,235683.0,176.0,89,4.64,45.0,-0.024403,13.0,down
3,2018-12,2018,Cherokee,199000.0,545.0,132.0,48.0,0.0,52.0,1708.0,264228.0,547.0,272.1,4.64,26.0,-0.004502,2.0,down
4,2018-12,2018,Orange,400000.0,364.0,105.0,80.0,0.0,56.0,2350.0,520135.0,445.0,153.6,4.64,117.0,-0.024390,126.0,down


In [78]:
# Create a new DataFrame that holds only the county names.
re_name_df = re_df[["County"]]
re_name_df.head()
len(re_name_df)

7196

In [79]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
df.drop(columns=["County", "Date", "Price_Status"], inplace=True)
df.head(10)

C:\Users\earth\anaconda3\envs\mlenv\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Year,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,average_listing_price,total_listing_count,violent_crime_rate,30_Year_Fixed_Rate,Number_of_Schools,median_listing_price_mm,Number_of_Parks
0,2018,224000.0,306.0,126.0,52.0,0.0,28.0,2174.0,352753.0,340.0,243.6,4.64,53.0,-0.025663,34.0
1,2018,245000.0,1197.0,85.0,296.0,12.0,204.0,2234.0,325987.0,1445.0,660.2,4.64,354.0,-0.019608,479.0
2,2018,179900.0,145.0,114.0,28.0,0.0,24.0,1987.0,235683.0,176.0,89,4.64,45.0,-0.024403,13.0
3,2018,199000.0,545.0,132.0,48.0,0.0,52.0,1708.0,264228.0,547.0,272.1,4.64,26.0,-0.004502,2.0
4,2018,400000.0,364.0,105.0,80.0,0.0,56.0,2350.0,520135.0,445.0,153.6,4.64,117.0,-0.024390,126.0
5,2018,192000.0,34.0,127.0,8.0,0.0,4.0,1786.0,207534.0,39.0,203.6,4.64,20.0,-0.038558,10.0
6,2018,439000.0,418.0,162.0,20.0,4.0,52.0,2265.0,644197.0,479.0,141.7,4.64,17.0,0.023310,17.0
7,2018,299990.0,689.0,60.0,216.0,68.0,248.0,2410.0,334948.0,904.0,127.1,4.64,165.0,-0.008298,120.0
8,2018,147000.0,124.0,103.0,20.0,0.0,16.0,1750.0,241071.0,132.0,479.4,4.64,32.0,-0.080675,15.0
9,2018,159000.0,2055.0,208.0,96.0,8.0,200.0,2059.0,313020.0,2253.0,62.2,4.64,40.0,-0.036072,44.0


In [80]:
# Use get_dummies() to create variables for text features.
x = pd.get_dummies(df, columns=["median_listing_price"])
x.head(10)

,Year,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,average_listing_price,total_listing_count,violent_crime_rate,...,median_listing_price_425000.0,median_listing_price_428903.0,median_listing_price_429000.0,median_listing_price_431950.0,median_listing_price_434900.0,median_listing_price_439000.0,median_listing_price_539900.0,median_listing_price_544400.0,median_listing_price_547000.0,median_listing_price_549000.0
0,2018,306.0,126.0,52.0,0.0,28.0,2174.0,352753.0,340.0,243.6,...,0,0,0,0,0,0,0,0,0,0
1,2018,1197.0,85.0,296.0,12.0,204.0,2234.0,325987.0,1445.0,660.2,...,0,0,0,0,0,0,0,0,0,0
2,2018,145.0,114.0,28.0,0.0,24.0,1987.0,235683.0,176.0,89,...,0,0,0,0,0,0,0,0,0,0
3,2018,545.0,132.0,48.0,0.0,52.0,1708.0,264228.0,547.0,272.1,...,0,0,0,0,0,0,0,0,0,0
4,2018,364.0,105.0,80.0,0.0,56.0,2350.0,520135.0,445.0,153.6,...,0,0,0,0,0,0,0,0,0,0
5,2018,34.0,127.0,8.0,0.0,4.0,1786.0,207534.0,39.0,203.6,...,0,0,0,0,0,0,0,0,0,0
6,2018,418.0,162.0,20.0,4.0,52.0,2265.0,644197.0,479.0,141.7,...,0,0,0,0,0,1,0,0,0,0
7,2018,689.0,60.0,216.0,68.0,248.0,2410.0,334948.0,904.0,127.1,...,0,0,0,0,0,0,0,0,0,0
8,2018,124.0,103.0,20.0,0.0,16.0,1750.0,241071.0,132.0,479.4,...,0,0,0,0,0,0,0,0,0,0
9,2018,2055.0,208.0,96.0,8.0,200.0,2059.0,313020.0,2253.0,62.2,...,0,0,0,0,0,0,0,0,0,0


In [81]:
# Standardize the data with StandardScaler().
re_scaled = StandardScaler().fit_transform(x)
re_scaled

array([[ 0.        , -0.46644102,  1.07565151, ..., -0.05471757,
        -0.05471757, -0.07749843],
       [ 0.        ,  0.78688434, -0.42501619, ..., -0.05471757,
        -0.05471757, -0.07749843],
       [ 0.        , -0.69291171,  0.6364317 , ..., -0.05471757,
        -0.05471757, -0.07749843],
       ...,
       [ 0.        ,  0.09903238, -0.7910327 , ..., -0.05471757,
        -0.05471757, -0.07749843],
       [ 0.        , -0.79278388,  0.6364317 , ..., -0.05471757,
        -0.05471757, -0.07749843],
       [ 0.        ,  0.530874  ,  0.67303335, ..., -0.05471757,
        -0.05471757, -0.07749843]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [82]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
re_pca = pca.fit_transform(re_scaled)
re_pca

array([[-1.30377533,  0.91721428,  0.66724973],
       [ 2.11385958, -1.42994833,  0.19041385],
       [-1.739669  ,  0.01266625,  0.1627845 ],
       ...,
       [ 0.13798098, -0.67439071, -1.50718118],
       [-1.80188101, -2.30540534,  0.40246723],
       [-0.01544627,  2.43431305,  0.49486072]])

In [83]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
            data=re_pca, columns=["PC 1", "PC 2", "PC 3"], index=df.index)
pcs_df.head(10)

,PC 1,PC 2,PC 3
0,-1.303775,0.917214,0.667250
1,2.113860,-1.429948,0.190414
2,-1.739669,0.012666,0.162785
3,-1.647413,-0.443298,1.428429
4,-0.132068,1.980880,0.094634
5,-2.261027,-0.433714,1.315653
6,-1.179188,3.675265,1.171427
7,1.241540,0.782527,-1.872327
8,-1.788980,-1.664521,1.143052
9,-0.109239,1.240578,3.411085


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [84]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Looking for the best K
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
re_elbow_df = pd.DataFrame(elbow_data)
re_elbow_df.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)


C:\Users\earth\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=3`

In [100]:
# Initialize the K-Means model.
model = KMeans(n_clusters=3, random_state=1)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
print(predictions)

[0 0 0 0 2 0 2 2 0 2 2 0 0 1 1 0 0 0 2 0 0 0 0 0 2 2 0 2 0 0 2 0 0 0 0 0 0
 0 0 2 0 0 0 2 0 0 0 0 0 0 0 0 2 0 0 0 2 0 2 2 0 2 2 2 0 0 0 0 0 0 0 0 0 2
 2 0 2 0 2 0 0 0 0 0 2 2 0 0 0 0 2 2 2 0 0 0 2 0 0 0 0 0 0 1 0 0 2 0 2 0 0
 0 2 0 2 0 1 0 0 2 2 0 0 0 0 0 2 0 0 2 0 2 2 0 0 0 0 0 0 2 2 0 0 0 2 0 0 0
 2 2 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 2 2 0 2 0 0 0 0 2 0 0 0 0 2 2 0 2 2
 2 0 1 0 0 0 2 0 1 0 0 0 2 0 2 0 2 2 0 0 0 1 0 0 2 0 2 0 0 0 0 0 0 0 2 0 0
 0 0 2 0 0 0 0 2 0 0 0 0 0 0 0 0 2 0 2 0 0 0 1 0 0 2 2 0 2 0 0 0 2 2 2 0 2
 0 0 0 0 2 0 2 0 0 0 0 0 0 0 0 2 0 2 0 0 0 2 2 0 1 0 2 0 0 2 1 2 0 0 0 2 0
 0 0 0 0 0 0 0 2 0 0 2 0 0 0 0 0 0 2 2 0 0 0 2 0 0 0 0 0 0 2 0 0 2 0 0 2 0
 0 2]


In [101]:
len(clustered_df)

7196

In [102]:
# Create a new DataFrame including predicted clusters and real estate features.
# Concatentate the df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([df, pcs_df], axis=1)

#  Add a new column, "County" to the clustered_df DataFrame that holds the names of the counties. 
clustered_df = pd.concat([clustered_df,re_name_df], axis=1)

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = pd.Series(predictions, index=df.index)

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(20)

(7196, 20)


,Year,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,average_listing_price,total_listing_count,violent_crime_rate,30_Year_Fixed_Rate,Number_of_Schools,median_listing_price_mm,Number_of_Parks,PC 1,PC 2,PC 3,County,Class
0,2018.0,224000.0,306.0,126.0,52.0,0.0,28.0,2174.0,352753.0,340.0,243.6,4.64,53.0,-0.025663,34.0,-1.303775,0.917214,0.667250,Cherokee,0.0
1,2018.0,245000.0,1197.0,85.0,296.0,12.0,204.0,2234.0,325987.0,1445.0,660.2,4.64,354.0,-0.019608,479.0,2.113860,-1.429948,0.190414,Craven,0.0
2,2018.0,179900.0,145.0,114.0,28.0,0.0,24.0,1987.0,235683.0,176.0,89,4.64,45.0,-0.024403,13.0,-1.739669,0.012666,0.162785,Catawba,0.0
3,2018.0,199000.0,545.0,132.0,48.0,0.0,52.0,1708.0,264228.0,547.0,272.1,4.64,26.0,-0.004502,2.0,-1.647413,-0.443298,1.428429,Gaston,0.0
4,2018.0,400000.0,364.0,105.0,80.0,0.0,56.0,2350.0,520135.0,445.0,153.6,4.64,117.0,-0.024390,126.0,-0.132068,1.980880,0.094634,Warren,2.0
5,2018.0,192000.0,34.0,127.0,8.0,0.0,4.0,1786.0,207534.0,39.0,203.6,4.64,20.0,-0.038558,10.0,-2.261027,-0.433714,1.315653,Lenoir,0.0
6,2018.0,439000.0,418.0,162.0,20.0,4.0,52.0,2265.0,644197.0,479.0,141.7,4.64,17.0,0.023310,17.0,-1.179188,3.675265,1.171427,Lee,2.0
7,2018.0,299990.0,689.0,60.0,216.0,68.0,248.0,2410.0,334948.0,904.0,127.1,4.64,165.0,-0.008298,120.0,1.241540,0.782527,-1.872327,Henderson,2.0
8,2018.0,147000.0,124.0,103.0,20.0,0.0,16.0,1750.0,241071.0,132.0,479.4,4.64,32.0,-0.080675,15.0,-1.788980,-1.664521,1.143052,Beaufort,0.0
9,2018.0,159000.0,2055.0,208.0,96.0,8.0,200.0,2059.0,313020.0,2253.0,62.2,4.64,40.0,-0.036072,44.0,-0.109239,1.240578,3.411085,Hertford,2.0


In [103]:
# adding "class" creates Nans b/c dataframes are different lengths, drop to graph.
clustered_df1 = clustered_df.dropna()


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [104]:
# Creating a 3D-Scatter with the PCA data and the clusters

fig = px.scatter_3d(
    clustered_df1,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    hover_data=["median_listing_price"],
    hover_name="County",
    color="Class",
    symbol="Class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

# kmeans=2

In [112]:
# Initialize the K-Means model.
model = KMeans(n_clusters=2, random_state=1)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)


In [111]:
# Create a new DataFrame including predicted clusters and real estate features.
# Concatentate the df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([df, pcs_df], axis=1)

#  Add a new column, "County" to the clustered_df DataFrame that holds the names of the counties. 
clustered_df = pd.concat([clustered_df,re_name_df], axis=1)

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = pd.Series(predictions, index=df.index)

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head()

(7196, 20)


,Year,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,average_listing_price,total_listing_count,violent_crime_rate,30_Year_Fixed_Rate,Number_of_Schools,median_listing_price_mm,Number_of_Parks,PC 1,PC 2,PC 3,County,Class
0,2018.0,224000.0,306.0,126.0,52.0,0.0,28.0,2174.0,352753.0,340.0,243.6,4.64,53.0,-0.025663,34.0,-1.303775,0.917214,0.667250,Cherokee,1.0
1,2018.0,245000.0,1197.0,85.0,296.0,12.0,204.0,2234.0,325987.0,1445.0,660.2,4.64,354.0,-0.019608,479.0,2.113860,-1.429948,0.190414,Craven,1.0
2,2018.0,179900.0,145.0,114.0,28.0,0.0,24.0,1987.0,235683.0,176.0,89,4.64,45.0,-0.024403,13.0,-1.739669,0.012666,0.162785,Catawba,1.0
3,2018.0,199000.0,545.0,132.0,48.0,0.0,52.0,1708.0,264228.0,547.0,272.1,4.64,26.0,-0.004502,2.0,-1.647413,-0.443298,1.428429,Gaston,1.0
4,2018.0,400000.0,364.0,105.0,80.0,0.0,56.0,2350.0,520135.0,445.0,153.6,4.64,117.0,-0.024390,126.0,-0.132068,1.980880,0.094634,Warren,1.0


In [110]:
# adding "class" creates Nans b/c dataframes are different lengths, drop to graph.
clustered_df1 = clustered_df.dropna()
# Creating a 3D-Scatter with the PCA data and the clusters

fig = px.scatter_3d(
    clustered_df1,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    hover_data=["median_listing_price"],
    hover_name="County",
    color="Class",
    symbol="Class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()